In [1]:
DATASET = 'crowd'

In [2]:
# Use output of OpenPose without background
# Paths to videos for training
PATHS = [f"../datasets/{DATASET}_dataset/original_data/", f"../datasets/{DATASET}_dataset/openpose_gamma/"]

FRAME_FUNC = 'frame_diff'
# To use frame diff to weight t (current) or t+1
WEIGHT_CURRENT = True

In [3]:
import tensorflow as tf

In [4]:
!export TF_FORCE_GPU_ALLOW_GROWTH=True
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0:1], 'GPU')

2022-07-23 18:59:10.409086: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 18:59:10.411096: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 18:59:10.412749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 18:59:10.418337: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 18:59:10.420379: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Set seed

In [5]:
SEED = 0
import random
import numpy as np
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Load data

In [6]:
import numpy as np
import cv2

In [7]:
FRAMES_PER_VIDEO = 50 + 1
VIDEO_WIDTH, VIDEO_HEIGHT = 100, 100
N_CHANNELS = 3

def load_videos(video_IDs: list, video_frames: int = FRAMES_PER_VIDEO, video_width: int = VIDEO_WIDTH, video_height: int = VIDEO_HEIGHT,
                video_channels: int = N_CHANNELS, dtype = np.float32, normalize: bool = False) -> tuple:
    videos = np.empty((len(video_IDs), video_frames, video_height, video_width, video_channels), dtype=dtype)

    for i, video_ID in enumerate(video_IDs):
        cap = cv2.VideoCapture(video_ID)
        original_n_frames = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))
        
        # Indexes of frames to be kept to comply with video_frames
        frames_idx = set(np.round(np.linspace(0, original_n_frames - 1, video_frames)).astype(int))

        frames = []
        index = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if index in frames_idx:
                frame = cv2.resize(frame, (video_width, video_height)).astype(dtype)
                if normalize:
                    frame /= 255.0
                frames.append(frame)
            index += 1
        cap.release()

        if len(frames) != video_frames:
            # Complete with repeated frames in the begging and the end of the video
            n_repeats = (video_frames - len(frames)) / 2
            # In case n_repeats is decimal, first frames will be rounded to the nearest integer
            beggining_frames = np.tile(frames[0], [round(n_repeats + 0.001), 1, 1, 1])
            end_frames = np.tile(frames[-1], [int(n_repeats), 1, 1, 1])
            frames = np.concatenate([beggining_frames, frames, end_frames])
        
        videos[i,] = np.array(frames)
        

    return videos

DataGenerator class to load videos per batch, in case all videos do not fit in memory

In [8]:
import gc
from tensorflow.keras import backend as K

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, video_IDs: list, video_labels: list, batch_size: int, paths: list = [''], video_width: int = VIDEO_WIDTH, video_height: int = VIDEO_HEIGHT,
                video_frames: int = FRAMES_PER_VIDEO, video_channels: int = N_CHANNELS, dtype = np.float32, normalize: bool = False, shuffle: bool = True):
        self.video_IDs = video_IDs
        self.video_labels = video_labels
        self.batch_size = batch_size
        self.paths = paths
        self.video_width = video_width
        self.video_height = video_height
        self.video_frames = video_frames
        self.video_channels = video_channels
        self.dtype = dtype
        self.normalize = normalize
        self.shuffle = shuffle

    def __len__(self):
        return len(self.video_IDs) // self.batch_size

    def __getitem__(self, idx):
        batch_IDs = self.video_IDs[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_labels = self.video_labels[idx*self.batch_size:(idx+1)*self.batch_size]
        input_videos = []
        
        for index, path in enumerate(self.paths):
            batch_IDs_full_path = [path+ID for ID in batch_IDs]

            videos = load_videos(batch_IDs_full_path, self.video_frames, self.video_width, 
                                         self.video_height, self.video_channels, self.dtype, self.normalize)
            
            input_videos.append(videos)
                    
        return input_videos, batch_labels
            
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            # shuffle video_IDs and video_labels together
            temp_list = list(zip(self.video_IDs, self.video_labels))
            np.random.shuffle(temp_list)
            self.video_IDs, self.video_labels = zip(*temp_list)
        # Clear memory after epochs
        gc.collect()
        #K.clear_session()

## Videos to load

In [9]:
import pandas as pd
folds = pd.read_csv(f'../datasets/{DATASET}_dataset/folds.csv')

def fecth_generators(fold):

    train_df = folds[folds.fold != fold]
    test_df = folds[folds.fold == fold]

    train_video_IDs = train_df.video.values
    test_video_IDs = test_df.video.values

    train_video_labels = train_df.label.values
    test_video_labels = test_df.label.values

    train_generator = DataGenerator(train_video_IDs, train_video_labels, batch_size=10, paths=PATHS)
    test_generator = DataGenerator(test_video_IDs, test_video_labels, batch_size=10, paths=PATHS)

    return train_generator, test_generator

# Frame functions

In [10]:
import tensorflow.keras.backend as K

def tf_frame_diff(video):
    return video[1:] - video[:-1]

frame_func = tf_frame_diff

# Load pretrained best model without optimizer

In [11]:
cv_acc = 0.0

for fold in range(5):
    print('#'*10, 'Fold', fold, '#'*10)

    model = tf.keras.models.load_model('models/rwf_best_model_no_optimizer.h5')
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'models/{DATASET}_best_model_fold{fold}.h5',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)

    train_generator, test_generator = fecth_generators(fold)
    history = model.fit(train_generator, epochs=50, validation_data=test_generator, verbose=2, callbacks=[model_checkpoint_callback])

    fold_acc = max(history.history['val_accuracy'])
    print('#'*10, f'Fold {fold} accuracy:', fold_acc, '#'*10)
    cv_acc += fold_acc / 5

########## Fold 0 ##########


2022-07-23 18:59:10.525827: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-23 18:59:10.528504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 18:59:10.530576: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 18:59:10.532600: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/50


2022-07-23 18:59:15.271468: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204
2022-07-23 18:59:15.890595: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-07-23 18:59:15.891858: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2022-07-23 18:59:15.891890: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-07-23 18:59:15.894487: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

19/19 - 18s - loss: 0.5961 - accuracy: 0.7526 - val_loss: 0.7428 - val_accuracy: 0.6800 - 18s/epoch - 936ms/step
Epoch 2/50
19/19 - 12s - loss: 0.4645 - accuracy: 0.7789 - val_loss: 0.4429 - val_accuracy: 0.8200 - 12s/epoch - 629ms/step
Epoch 3/50
19/19 - 12s - loss: 0.3795 - accuracy: 0.8474 - val_loss: 0.4022 - val_accuracy: 0.8200 - 12s/epoch - 629ms/step
Epoch 4/50
19/19 - 12s - loss: 0.3409 - accuracy: 0.8684 - val_loss: 0.4530 - val_accuracy: 0.8000 - 12s/epoch - 629ms/step
Epoch 5/50
19/19 - 12s - loss: 0.3426 - accuracy: 0.8632 - val_loss: 0.2919 - val_accuracy: 0.8800 - 12s/epoch - 631ms/step
Epoch 6/50
19/19 - 12s - loss: 0.2892 - accuracy: 0.8684 - val_loss: 0.2717 - val_accuracy: 0.9000 - 12s/epoch - 631ms/step
Epoch 7/50
19/19 - 12s - loss: 0.3406 - accuracy: 0.8842 - val_loss: 0.1946 - val_accuracy: 0.9400 - 12s/epoch - 629ms/step
Epoch 8/50
19/19 - 12s - loss: 0.2777 - accuracy: 0.8947 - val_loss: 0.2898 - val_accuracy: 0.8600 - 12s/epoch - 629ms/step
Epoch 9/50
19/19 - 

19/19 - 11s - loss: 0.1364 - accuracy: 0.9474 - val_loss: 0.6725 - val_accuracy: 0.8250 - 11s/epoch - 597ms/step
Epoch 16/50
19/19 - 11s - loss: 0.1458 - accuracy: 0.9368 - val_loss: 0.5805 - val_accuracy: 0.8500 - 11s/epoch - 597ms/step
Epoch 17/50
19/19 - 11s - loss: 0.1198 - accuracy: 0.9579 - val_loss: 0.6614 - val_accuracy: 0.8250 - 11s/epoch - 593ms/step
Epoch 18/50
19/19 - 11s - loss: 0.1186 - accuracy: 0.9632 - val_loss: 0.5221 - val_accuracy: 0.8750 - 11s/epoch - 601ms/step
Epoch 19/50
19/19 - 11s - loss: 0.1291 - accuracy: 0.9579 - val_loss: 0.6059 - val_accuracy: 0.8500 - 11s/epoch - 599ms/step
Epoch 20/50
19/19 - 11s - loss: 0.1618 - accuracy: 0.9526 - val_loss: 0.7427 - val_accuracy: 0.8500 - 11s/epoch - 597ms/step
Epoch 21/50
19/19 - 11s - loss: 0.0786 - accuracy: 0.9842 - val_loss: 0.8132 - val_accuracy: 0.8000 - 11s/epoch - 598ms/step
Epoch 22/50
19/19 - 11s - loss: 0.0548 - accuracy: 0.9789 - val_loss: 0.8714 - val_accuracy: 0.8250 - 11s/epoch - 595ms/step
Epoch 23/50


19/19 - 11s - loss: 0.0997 - accuracy: 0.9474 - val_loss: 0.1352 - val_accuracy: 0.9750 - 11s/epoch - 602ms/step
Epoch 30/50
19/19 - 11s - loss: 0.0858 - accuracy: 0.9632 - val_loss: 0.0360 - val_accuracy: 1.0000 - 11s/epoch - 601ms/step
Epoch 31/50
19/19 - 11s - loss: 0.0810 - accuracy: 0.9684 - val_loss: 0.3930 - val_accuracy: 0.9250 - 11s/epoch - 600ms/step
Epoch 32/50
19/19 - 11s - loss: 0.0976 - accuracy: 0.9526 - val_loss: 0.2779 - val_accuracy: 0.9500 - 11s/epoch - 596ms/step
Epoch 33/50
19/19 - 11s - loss: 0.0807 - accuracy: 0.9579 - val_loss: 0.3252 - val_accuracy: 0.9500 - 11s/epoch - 600ms/step
Epoch 34/50
19/19 - 11s - loss: 0.0862 - accuracy: 0.9632 - val_loss: 0.3197 - val_accuracy: 0.9500 - 11s/epoch - 599ms/step
Epoch 35/50
19/19 - 11s - loss: 0.0907 - accuracy: 0.9684 - val_loss: 0.3210 - val_accuracy: 0.9500 - 11s/epoch - 601ms/step
Epoch 36/50
19/19 - 11s - loss: 0.0566 - accuracy: 0.9842 - val_loss: 0.2828 - val_accuracy: 0.9250 - 11s/epoch - 595ms/step
Epoch 37/50


Epoch 44/50
19/19 - 12s - loss: 0.0672 - accuracy: 0.9737 - val_loss: 0.3336 - val_accuracy: 0.9250 - 12s/epoch - 610ms/step
Epoch 45/50
19/19 - 12s - loss: 0.1351 - accuracy: 0.9474 - val_loss: 0.4723 - val_accuracy: 0.9000 - 12s/epoch - 609ms/step
Epoch 46/50
19/19 - 12s - loss: 0.0888 - accuracy: 0.9632 - val_loss: 0.3093 - val_accuracy: 0.9000 - 12s/epoch - 607ms/step
Epoch 47/50
19/19 - 12s - loss: 0.0732 - accuracy: 0.9789 - val_loss: 0.3615 - val_accuracy: 0.9000 - 12s/epoch - 609ms/step
Epoch 48/50
19/19 - 12s - loss: 0.0434 - accuracy: 0.9895 - val_loss: 0.6022 - val_accuracy: 0.8500 - 12s/epoch - 609ms/step
Epoch 49/50
19/19 - 12s - loss: 0.0423 - accuracy: 0.9842 - val_loss: 0.5624 - val_accuracy: 0.8750 - 12s/epoch - 609ms/step
Epoch 50/50
19/19 - 12s - loss: 0.0414 - accuracy: 0.9895 - val_loss: 0.4350 - val_accuracy: 0.8750 - 12s/epoch - 610ms/step
########## Fold 3 accuracy: 0.925000011920929 ##########
########## Fold 4 ##########
Epoch 1/50
19/19 - 15s - loss: 0.7321 -

In [12]:
print('CV accuracy:', cv_acc)

CV accuracy: 0.9430000066757203
